LFIRE

[Efficient Bayesian Experimental Design for Implicit Models](https://arxiv.org/pdf/1810.09912)

[Likelihood-free inference by ratio estimation](https://arxiv.org/pdf/1611.10242)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy as sp
import seaborn as sns


In [2]:
data = np.load("deathmodel_dim1.npz")
pp = data['prior_samples']
y_obs = data["y_obs"]
data

NpzFile 'deathmodel_dim1.npz' with keys: d_opt, y_obs, r_obs, b_obs, utilobj...

In [3]:
#True theta value assumption made for the sake of simulation
true_theta = 0.5

#Sampling lots of prior theta values from the given prior distribution
theta_prior_samples = sp.stats.norm.rvs(0,1,size=100)



theta_prior_samples

array([-0.58196256,  0.39538354, -0.21808343,  0.19456922,  0.67764806,
       -0.33442919,  0.74542425,  0.04733211, -0.14998026,  0.28023439,
       -0.17558073,  0.33880205, -0.00553748, -0.05495615,  0.12827177,
        0.64273127, -0.57739727, -0.79360133, -0.77204657, -1.28721327,
        0.89985853,  1.25449158, -0.02677479,  1.0590545 , -0.0610058 ,
       -1.84452501,  0.90575919,  1.33986316,  1.46453887,  0.12885711,
       -1.12763088,  2.16885458,  1.84880027,  0.64073772,  1.96001498,
       -0.63366441,  1.86842778,  1.46245292,  0.9902182 , -0.20289468,
        2.03160321, -0.61079813,  0.0249167 ,  0.29526217,  1.01810722,
       -0.53602378,  0.26537878, -2.19910065,  1.20776018, -0.43712777,
        0.05481935,  0.45346205,  1.16798629,  1.04586903,  1.07855028,
       -1.19599884, -0.01844453, -0.03203053, -0.9219183 ,  1.59456229,
        0.39866392, -0.99848069,  1.2270882 ,  0.64496909,  0.56808113,
        0.55476264,  1.56040431, -0.27331293,  0.20000756, -0.31

In [8]:
import numpy as np
import warnings

class Simulator:

    """
    Simulator base class for simulating data from different models.
    """

    def __init__(self, truth):
        """
        truth: Ground truth that is used in the observe() method. Needs to be same dimensions and shape as the model parameters.
        """
        self.truth = np.array(truth)

    def summary(self, Y):

        """
        Method to take summary statistics of the simulated data. Default is simply powers from 1 to 4 of the data values; this is only applicable to scalars.
        Y: Data simulated from the model.
        """

        Y_psi = list()
        for y in Y:
            # could change 5 to any kind of degree
            Y_psi.append([y ** i for i in range(1, 5)])
        return np.array(Y_psi)

    def generate_data(self, d, p):

        """
        Child class specific method to simulate data from the model.
        d: design variable
        p: model parameters
        """

        pass

    def sample_data(self, d, p, num=None):

        """
        Sample data from the model, based on the generate_data() method. The point of this method is to select if to sample from the likelihood (if num!=None and len(p)==1) or marginal (if num==None and len(p) > 1).
        d: design variable
        p: model parameters
        num: number of samples required from likelihood
        """

        # sample from an array of params
        if num==None:
            y = np.array([self.generate_data(d, pi) for pi in p])
        # sample several times using the same params:
        else:
            y = np.array([self.generate_data(d, p) for i in range(num)])
        return y

    def observe(self, d, num=1):

        """
        Observe some data according to a ground truth.
        d: design variable (optimal)
        num: number of data points to observe at optimal design
        """

        y = np.array([self.generate_data(d, self.truth) for i in range(num)])
        return y


class DeathModel(Simulator):

    """
    Class to simulate data according to the Death Model.
    """

    def __init__(self, truth, S0):

        """
        truth: ground truth, scalar
        S0: starting population of death model
        """

        super(DeathModel, self).__init__(truth)
        self.S0 = S0

    def summary(self, Y):

        Y_psi = list()
        for arr in Y:
            if np.array(arr).shape == ():
                tmp = [arr, 0]
                #tmp = [arr ** i for i in range(1,5)]
            else:
                tmp = [arr[0], 0]
                #tmp = [arr[0] ** i for i in range(1, 5)]
            Y_psi.append(tmp)
        return np.array(Y_psi)

    def generate_data(self, d, p):

        """
        d: design (scalar)
        p: model parameter (scalar)
        """

        inf_prob = lambda b, t: 1 - np.exp(-b*t)
        print(inf_prob)
        inf_num = np.random.binomial(self.S0, inf_prob(p, d))
        return inf_num
    

model = DeathModel(1.5, 50)

print(model.generate_data(1,3))

<function DeathModel.generate_data.<locals>.<lambda> at 0x1475d2980>
45
